In [ ]:
import csv
import json

def generate_html(csv_file, output_file='roselia_songs.html'):
    # CSV 파일 읽기
    songs = []
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row.get('제목(원어)'):
                # 가사(콜표) 합치기
                lyrics1 = row.get('가사(콜표)', '').strip()
                lyrics2 = row.get('가사(콜표) 2', '').strip()
                row['가사(콜표)'] = lyrics1 + lyrics2
                songs.append(row)
    
    # JavaScript용 데이터 준비
    songs_json = json.dumps(songs, ensure_ascii=False)
    
    # HTML 템플릿
    html_template = f'''<!DOCTYPE html>
<html lang="ko">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Roselia 내한대비 곡 리스트</title>
  <style>
    * {{
      margin: 0;
      padding: 0;
      box-sizing: border-box;
    }}
    
    body {{
      font-family: 'Malgun Gothic', sans-serif;
      background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
      min-height: 100vh;
      padding: 20px;
    }}
    
    .container {{
      max-width: 1400px;
      margin: 0 auto;
      background: white;
      border-radius: 15px;
      box-shadow: 0 20px 60px rgba(0,0,0,0.3);
      overflow: hidden;
    }}
    
    header {{
      background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
      color: white;
      padding: 30px;
      text-align: center;
    }}
    
    .header-logo {{
      max-width: 400px;
      height: auto;
      margin: 0 auto 15px;
      display: block;
    }}
    
    h1 {{
      font-size: 2.5em;
      margin-bottom: 10px;
    }}
    
    .controls {{
      padding: 20px 30px;
      background: #f8f9fa;
      border-bottom: 2px solid #e9ecef;
      display: flex;
      gap: 15px;
      align-items: center;
    }}
    
    .controls label {{
      font-weight: 600;
      color: #667eea;
    }}
    
    .controls select {{
      padding: 8px 12px;
      border: 2px solid #667eea;
      border-radius: 5px;
      background: white;
      cursor: pointer;
      font-size: 14px;
    }}
    
    .table-wrapper {{
      overflow-x: auto;
    }}
    
    table {{
      width: 100%;
      border-collapse: collapse;
    }}
    
    thead {{
      background: #667eea;
      color: white;
    }}
    
    th {{
      padding: 15px;
      text-align: left;
      font-weight: 600;
      white-space: nowrap;
    }}
    
    td {{
      padding: 15px;
      border-bottom: 1px solid #e9ecef;
    }}
    
    tr:hover {{
      background: #f8f9fa;
    }}
    
    .tour-badge {{
      display: inline-block;
      background: #667eea;
      color: white;
      padding: 4px 8px;
      border-radius: 4px;
      font-size: 12px;
      font-weight: 600;
    }}
    
    .youtube-link {{
      color: #ff0000;
      text-decoration: none;
      font-weight: 600;
      transition: opacity 0.2s;
    }}
    
    .youtube-link:hover {{
      opacity: 0.7;
    }}
    
    .lyrics-btn {{
      background: #667eea;
      color: white;
      border: none;
      padding: 8px 16px;
      border-radius: 5px;
      cursor: pointer;
      font-weight: 600;
      transition: background 0.2s;
    }}
    
    .lyrics-btn:hover {{
      background: #764ba2;
    }}
    
    .modal {{
      display: none;
      position: fixed;
      top: 0;
      left: 0;
      width: 100%;
      height: 100%;
      background: rgba(0,0,0,0.7);
      z-index: 1000;
      overflow: auto;
    }}
    
    .modal-content {{
      background: white;
      margin: 50px auto;
      padding: 30px;
      max-width: 900px;
      border-radius: 15px;
      position: relative;
      max-height: 80vh;
      overflow-y: auto;
    }}
    
    .close-btn {{
      position: absolute;
      top: 15px;
      right: 20px;
      font-size: 30px;
      font-weight: bold;
      color: #666;
      cursor: pointer;
      transition: color 0.2s;
    }}
    
    .close-btn:hover {{
      color: #000;
    }}
    
    .modal-title {{
      color: #667eea;
      margin-bottom: 20px;
      padding-right: 40px;
    }}
    
    .lyrics-guide {{
      background: #f8f9fa;
      padding: 20px;
      border-radius: 10px;
      margin-bottom: 25px;
      border-left: 4px solid #667eea;
    }}
    
    .lyrics-guide p {{
      margin: 8px 0;
      line-height: 1.6;
    }}
    
    .lyrics-content {{
      line-height: 1.8;
    }}
  </style>
</head>
<body>
  <div class="container">
    <header>
      <img src="https://static.wikia.nocookie.net/bandori/images/a/aa/Roselia_logo.png/revision/latest?cb=20211201105701" alt="Roselia Logo" class="header-logo" onerror="this.style.display='none'">
      <h1>🎸 Roselia 내한대비 곡 리스트</h1>
      <p>곡 정보를 확인하고 가사를 감상하세요</p>
    </header>
    
    <div class="controls">
      <label for="sortSelect">정렬:</label>
      <select id="sortSelect" onchange="sortTable(this.value)">
        <option value="default">기본 순서</option>
        <option value="date-desc">발매일 (최신순)</option>
        <option value="date-asc">발매일 (오래된순)</option>
        <option value="rate-desc">선곡률 (높은순)</option>
        <option value="rate-asc">선곡률 (낮은순)</option>
        <option value="osaka">투어 오사카 (O)</option>
        <option value="singapore">투어 싱가폴 (O)</option>
      </select>
    </div>
    
    <div class="table-wrapper">
      <div id="tableContainer"></div>
    </div>
  </div>
  
  <div class="modal" id="lyricsModal">
    <div class="modal-content">
      <span class="close-btn" onclick="closeModal()">&times;</span>
      <h2 class="modal-title" id="modalTitle"></h2>
      
      <div class="lyrics-guide">
        <p><span style="background-color: rgb(255, 198, 156); padding: 2px 8px; border-radius: 3px;">주황색 배경 ★하이콜</span> : 하이 하이 혹은 어이 어이 라고 외치는 그것...</p>
        <p><span style="background-color: rgb(206, 231, 247); padding: 2px 8px; border-radius: 3px;">하늘색 배경</span> : 박수</p>
        <p><b style="color: #ff0000;">붉은색 굵은글씨</b> : 중요도가 높은 콜</p>
        <p style="color: #ff0000;">붉은색 얇은글씨 : 중요도가 낮거나, 따라부르듯이 하는 콜</p>
      </div>
      
      <div class="lyrics-content" id="lyricsContent"></div>
    </div>
  </div>
  
  <script>
    let songsData = {songs_json};
    let currentSort = 'default';
    let displayedSongs = [...songsData];
    
    function sortTable(sortType) {{
      currentSort = sortType;
      
      switch(sortType) {{
        case 'date-desc':
          displayedSongs.sort((a, b) => {{
            const dateA = new Date(a['발매일'] || '1900-01-01');
            const dateB = new Date(b['발매일'] || '1900-01-01');
            return dateB - dateA;
          }});
          break;
        case 'date-asc':
          displayedSongs.sort((a, b) => {{
            const dateA = new Date(a['발매일'] || '1900-01-01');
            const dateB = new Date(b['발매일'] || '1900-01-01');
            return dateA - dateB;
          }});
          break;
        case 'rate-desc':
          displayedSongs.sort((a, b) => {{
            const rateA = parseFloat(a['과거기반 선곡률'] || 0);
            const rateB = parseFloat(b['과거기반 선곡률'] || 0);
            return rateB - rateA;
          }});
          break;
        case 'rate-asc':
          displayedSongs.sort((a, b) => {{
            const rateA = parseFloat(a['과거기반 선곡률'] || 0);
            const rateB = parseFloat(b['과거기반 선곡률'] || 0);
            return rateA - rateB;
          }});
          break;
        case 'osaka':
          displayedSongs = songsData.filter(song => song['투어_오사카'] === '1');
          break;
        case 'singapore':
          displayedSongs = songsData.filter(song => song['투어_싱가폴'] === '1');
          break;
        default:
          displayedSongs = [...songsData];
      }}
      
      displayTable();
    }}
    
    function displayTable() {{
      const container = document.getElementById('tableContainer');
      
      let html = `
        <table>
          <thead>
            <tr>
              <th>제목</th>
              <th>발매일</th>
              <th>선곡률</th>
              <th>투어</th>
              <th>유튜브</th>
              <th>가사</th>
            </tr>
          </thead>
          <tbody>
      `;
      
      displayedSongs.forEach((song, index) => {{
        // 제목 통합
        let title = song['제목(원어)'] || '';
        if (song['제목(한국어)']) {{
          title += ` (${{song['제목(한국어)']}})`;
        }}
        
        // 투어 정보
        let tourInfo = [];
        if (song['투어_오사카'] === '1') {{
          tourInfo.push('<span class="tour-badge">오사카</span>');
        }}
        if (song['투어_싱가폴'] === '1') {{
          tourInfo.push('<span class="tour-badge">싱가폴</span>');
        }}
        const tourDisplay = tourInfo.length > 0 ? tourInfo.join(' ') : '-';
        
        html += `
          <tr>
            <td>${{title || '-'}}</td>
            <td>${{song['발매일'] || '-'}}</td>
            <td>${{song['과거기반 선곡률'] || '-'}}%</td>
            <td>${{tourDisplay}}</td>
            <td>
              ${{song['유튜브'] ? `<a href="${{song['유튜브']}}" class="youtube-link" target="_blank">▶ 유튜브</a>` : '-'}}
            </td>
            <td>
              <button class="lyrics-btn" onclick="showLyrics(${{index}})">가사 보기</button>
            </td>
          </tr>
        `;
      }});
      
      html += '</tbody></table>';
      container.innerHTML = html;
    }}
    
    function showLyrics(index) {{
      const song = displayedSongs[index];
      const modal = document.getElementById('lyricsModal');
      const title = document.getElementById('modalTitle');
      const content = document.getElementById('lyricsContent');
      
      let titleText = song['제목(원어)'] || '';
      if (song['제목(한국어)']) {{
        titleText += ` (${{song['제목(한국어)']}})`;
      }}
      
      title.textContent = titleText;
      content.innerHTML = song['가사(콜표)'] || '가사 정보가 없습니다.';
      
      modal.style.display = 'block';
    }}
    
    function closeModal() {{
      document.getElementById('lyricsModal').style.display = 'none';
    }}
    
    window.onclick = function(event) {{
      const modal = document.getElementById('lyricsModal');
      if (event.target === modal) {{
        closeModal();
      }}
    }}
    
    // 페이지 로드 시 테이블 표시
    displayTable();
  </script>
</body>
</html>'''
    
    # HTML 파일 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_template)
    
    print(f"✅ HTML 파일이 생성되었습니다: {output_file}")
    print(f"📊 총 {len(songs)}곡의 데이터가 포함되었습니다.")
    print(f"\n💡 팁: 로고 이미지를 사용하려면 HTML 파일의 img src를 수정하세요.")

# 사용 예시
if __name__ == "__main__":
    # CSV 파일명을 여기에 입력하세요
    csv_filename = "roselia.csv"
    
    try:
        generate_html(csv_filename)
    except FileNotFoundError:
        print(f"❌ 오류: '{csv_filename}' 파일을 찾을 수 없습니다.")
        print("CSV 파일과 이 스크립트를 같은 폴더에 두고 실행해주세요.")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

✅ HTML 파일이 생성되었습니다: roselia_songs.html
📊 총 56곡의 데이터가 포함되었습니다.
